# Le PIB est-il un indicateur robuste de l'accès à l'alimentation ?

In [56]:
pip install faostat

Note: you may need to restart the kernel to use updated packages.


In [57]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Partie 1 : Collecte des données

### Récupération des données de PIB à partir de la base de données de la banque mondiale

In [58]:
url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.KD?format=json&per_page=20000"


response = requests.get(url)

# Extraction du JSON de la réponse
data = response.json()  # ici c’est correct

# Vérification des données
print(type(data),len(data))
print(data[0]) # Métadonnées
print(data[1][:5]) # Données de PIB par pays

data = data[1]  # On ne garde pas les métadonnées



<class 'list'> 2
{'page': 1, 'pages': 1, 'per_page': 20000, 'total': 17290, 'sourceid': '2', 'lastupdated': '2025-12-19'}
[{'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2024', 'value': 1435.35627774336, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2023', 'value': 1431.72178996886, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2022', 'value': 1440.43036630175, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 

#Retraitement des données


Création du DataFrame

In [59]:
from import_worldbank import parse_worldbank_pib

In [60]:
# On transforme le JSON en DataFrame et on ne garde que le PIB/hab pour chaque années avec le nom et le code du pays
data_worldbank = parse_worldbank_pib(data)
print(data_worldbank.head(10))

                          Pays  Année       Valeur
0  Africa Eastern and Southern   2024  1435.356278
1  Africa Eastern and Southern   2023  1431.721790
2  Africa Eastern and Southern   2022  1440.430366
3  Africa Eastern and Southern   2021  1425.209057
4  Africa Eastern and Southern   2020  1399.398498
5  Africa Eastern and Southern   2019  1479.372387
6  Africa Eastern and Southern   2018  1489.942137
7  Africa Eastern and Southern   2017  1490.911098
8  Africa Eastern and Southern   2016  1490.907546
9  Africa Eastern and Southern   2015  1498.875240


Filtrage des années à garder

In [61]:
from import_worldbank import filter_years

In [62]:
data_worldbank = filter_years(data_worldbank,2000,2020)
print(data_worldbank.head(10))

                           Pays  Année       Valeur
4   Africa Eastern and Southern   2020  1399.398498
5   Africa Eastern and Southern   2019  1479.372387
6   Africa Eastern and Southern   2018  1489.942137
7   Africa Eastern and Southern   2017  1490.911098
8   Africa Eastern and Southern   2016  1490.907546
9   Africa Eastern and Southern   2015  1498.875240
10  Africa Eastern and Southern   2014  1495.955574
11  Africa Eastern and Southern   2013  1478.977977
12  Africa Eastern and Southern   2012  1456.722813
13  Africa Eastern and Southern   2011  1468.630572


Restriction aux pays suivants : France, Etats-unis, Angleterre, Russie, Chine, Japon, Brésil, Afrique du Sud

In [63]:
from import_worldbank import filter_bricsam_countries

In [64]:
pays_a_conserver = [
        'India', 'China', 'Brazil', 'South Africa', 
        'Russian Federation', 'Mexico'
    ]

data_worldbank = filter_bricsam_countries(data_worldbank, pays_a_conserver)

# Vérifier le résultat
print(data_worldbank['Pays'].unique())
print(data_worldbank.head(10))

['Brazil' 'China' 'India' 'Mexico' 'Russian Federation' 'South Africa']
        Pays  Année       Valeur
4879  Brazil   2020  8435.010462
4880  Brazil   2019  8771.439504
4881  Brazil   2018  8722.335300
4882  Brazil   2017  8628.252096
4883  Brazil   2016  8577.842780
4884  Brazil   2015  8936.195589
4885  Brazil   2014  9338.341708
4886  Brazil   2013  9366.738247
4887  Brazil   2012  9167.498185
4888  Brazil   2011  9067.992120


Gestion des valeurs manquantes

In [65]:
for pays in data_worldbank['Pays'].unique() :
    print(data_worldbank.loc[data_worldbank['Pays'] == pays].isna().sum(),pays)

Pays      0
Année     0
Valeur    0
dtype: int64 Brazil
Pays      0
Année     0
Valeur    0
dtype: int64 China
Pays      0
Année     0
Valeur    0
dtype: int64 India
Pays      0
Année     0
Valeur    0
dtype: int64 Mexico
Pays      0
Année     0
Valeur    0
dtype: int64 Russian Federation
Pays      0
Année     0
Valeur    0
dtype: int64 South Africa


### Récupération des données indicateurs alimentaires à partir de la base de données FAOSTAT

In [66]:
import faostat

df = faostat.list_datasets_df() # cette fonction permet de récupérer l'ensemble des indicateurs disponible
                                # ainsi que d'avoir des informations sur les màj de versions, etc
print(df)

   code                              label date_update     note_update  \
0   QCL       Crops and livestock products  2025-06-11  minor revision   
1    QI                 Production Indices  2025-02-03                   
2    QV   Value of Agricultural Production  2025-02-03                   
3    FS  Suite of Food Security Indicators  2025-07-28                   
4   FBS              Food Balances (2010-)  2025-10-28                   
..  ...                                ...         ...             ...   
63   FA           Food Aid Shipments (WFP)  2016-12-22                   
64   RM                          Machinery  2021-12-03  minor revision   
65   RY                  Machinery Archive  2021-12-03  minor revision   
66   RA                Fertilizers archive  2020-09-08  minor revision   
67   PA       Producer Prices (old series)  1991-12-31                   

            release_current state_current year_current release_next  \
0   2024-12-20 / 2025-06-11         fina

On utilise la fonction définie dans le fichier py import des données faostat pour récupérer les données de 4 indicateurs  pour les pays des BRICSAM sur la période 2000 à 2022.
Les indicateurs sont :
"suffisance des apports énergétiques alimentaires moyens"
"disponibilité alimentaire par habitant"
"disponibilité protéiques moyenne"


In [67]:
from import_fao import fetch_fao_security_data

In [68]:

liste_pays = [202,185,138,100,351,21] # Correspond aux codes des pays des BRICSAM
liste_indicateurs = ['21010','22000','21013'] # Correspond aux codes des indicateurs d'insécurité alimentaire
liste_années = [2000, 2020] # Correspond à la page de dates.

data_faostat = fetch_fao_security_data(liste_pays, liste_indicateurs, liste_années)

print(data_faostat.head(10))



  Domain Code                             Domain  Area Code    Area  \
0          FS  Suite of Food Security Indicators         21  Brazil   
1          FS  Suite of Food Security Indicators         21  Brazil   
2          FS  Suite of Food Security Indicators         21  Brazil   
3          FS  Suite of Food Security Indicators         21  Brazil   
4          FS  Suite of Food Security Indicators         21  Brazil   
5          FS  Suite of Food Security Indicators         21  Brazil   
6          FS  Suite of Food Security Indicators         21  Brazil   
7          FS  Suite of Food Security Indicators         21  Brazil   
8          FS  Suite of Food Security Indicators         21  Brazil   
9          FS  Suite of Food Security Indicators         21  Brazil   

   Element Code Element  Item Code  \
0          6121   Value      21010   
1          6121   Value      21010   
2          6121   Value      21010   
3          6121   Value      21010   
4          6121   Value     

On vérifie ensuite qu'il n'y a pas de valeurs manquantes dans nos données. Comme ce n'est pas le cas, nous n'avons pas besoin de passer par un retraitement.

In [69]:
print(data_faostat.isna().sum())

Domain Code     0
Domain          0
Area Code       0
Area            0
Element Code    0
Element         0
Item Code       0
Item            0
Year Code       0
Year            0
Unit            0
Value           0
dtype: int64


Une fois que l'on a récupéré les données qui nous intéressent et avoir vérifié qu'il n'y avait pas de valeurs manquantes dans nos données, on passe par un retraitement des données (renommage des colonnes, supression de certaines, etc.)

In [70]:
from import_fao import clean_fao_columns

In [71]:
data_faostat = clean_fao_columns(data_faostat)
print(data_faostat.head(10))


     Pays                                         Indicateur      Année Unité  \
0  Brazil  Average dietary energy supply adequacy (percen...  2000-2002     %   
1  Brazil  Average dietary energy supply adequacy (percen...  2001-2003     %   
2  Brazil  Average dietary energy supply adequacy (percen...  2002-2004     %   
3  Brazil  Average dietary energy supply adequacy (percen...  2003-2005     %   
4  Brazil  Average dietary energy supply adequacy (percen...  2004-2006     %   
5  Brazil  Average dietary energy supply adequacy (percen...  2005-2007     %   
6  Brazil  Average dietary energy supply adequacy (percen...  2006-2008     %   
7  Brazil  Average dietary energy supply adequacy (percen...  2007-2009     %   
8  Brazil  Average dietary energy supply adequacy (percen...  2008-2010     %   
9  Brazil  Average dietary energy supply adequacy (percen...  2009-2011     %   

   Valeur  
0   122.0  
1   124.0  
2   127.0  
3   129.0  
4   130.0  
5   131.0  
6   132.0  
7   133.0  


Ensuite on traduit en français le nom des indicateurs pour faciliter les manipulations par la suite

In [72]:
from import_fao import translate_indicators

In [73]:
data_faostat = translate_indicators(data_faostat)
print(data_faostat.head(10))


     Pays                                         Indicateur      Année Unité  \
0  Brazil  suffisance des apports énergétiques alimentair...  2000-2002     %   
1  Brazil  suffisance des apports énergétiques alimentair...  2001-2003     %   
2  Brazil  suffisance des apports énergétiques alimentair...  2002-2004     %   
3  Brazil  suffisance des apports énergétiques alimentair...  2003-2005     %   
4  Brazil  suffisance des apports énergétiques alimentair...  2004-2006     %   
5  Brazil  suffisance des apports énergétiques alimentair...  2005-2007     %   
6  Brazil  suffisance des apports énergétiques alimentair...  2006-2008     %   
7  Brazil  suffisance des apports énergétiques alimentair...  2007-2009     %   
8  Brazil  suffisance des apports énergétiques alimentair...  2008-2010     %   
9  Brazil  suffisance des apports énergétiques alimentair...  2009-2011     %   

   Valeur  
0   122.0  
1   124.0  
2   127.0  
3   129.0  
4   130.0  
5   131.0  
6   132.0  
7   133.0  


Les indicateurs ont été moyennés sur 3 ans, et non par année. Nous avons des moyennes pour des périodes comme 2000-2002 et 2001-2003. Pour modéliser les données du PIB par habitant, nous avons décidé d'attribuer chaque moyenne triennale à la première année. Par exemple, la moyenne pour 2011-2013 correspond à 2011. Cela permet de maintenir la cohérence des tendances. Toutefois, si le PIB par habitant subit un choc au cours d'une année, le calcul de la moyenne peut atténuer ce choc sur les indicateurs alimentaires, ce qui affaiblit les résultats.

In [74]:
from import_fao import simplify_fao_years

In [75]:
data_faostat = simplify_fao_years(data_faostat)
print(data_faostat.head(10))


     Pays                                         Indicateur  Année Unité  \
0  Brazil  suffisance des apports énergétiques alimentair...   2000     %   
1  Brazil  suffisance des apports énergétiques alimentair...   2001     %   
2  Brazil  suffisance des apports énergétiques alimentair...   2002     %   
3  Brazil  suffisance des apports énergétiques alimentair...   2003     %   
4  Brazil  suffisance des apports énergétiques alimentair...   2004     %   
5  Brazil  suffisance des apports énergétiques alimentair...   2005     %   
6  Brazil  suffisance des apports énergétiques alimentair...   2006     %   
7  Brazil  suffisance des apports énergétiques alimentair...   2007     %   
8  Brazil  suffisance des apports énergétiques alimentair...   2008     %   
9  Brazil  suffisance des apports énergétiques alimentair...   2009     %   

   Valeur  
0   122.0  
1   124.0  
2   127.0  
3   129.0  
4   130.0  
5   131.0  
6   132.0  
7   133.0  
8   134.0  
9   136.0  


### Export du DataFrame final

Finalement, on va fusionner les deux dtaframes (worldbanket faostat) puis l'exporter en csv

In [76]:
def format_data_worldbank(data):
    """
    Harmonise le dataframe World Bank pour qu'il ait la même structure que FAOSTAT.
    """
    
    # 1. Ajouter les colonnes manquantes avec les valeurs fixes demandées
    data['Indicateur'] = "PIB par habitant"
    data['Unité'] = "$"
    
    # 2. Réorganiser l'ordre des colonnes pour qu'il soit identique à FAOSTAT
    return data[['Pays', 'Indicateur', 'Unité', 'Année', 'Valeur']]

In [77]:
# 1. On formate les données World Bank
data_worldbank_formatted = format_data_worldbank(data_worldbank)

# 2. On empile les deux DataFrames
# On utilise concat car on ajoute des nouveaux types d'indicateurs (lignes)
data_BRICSAM = pd.concat([data_faostat, data_worldbank_formatted], ignore_index=True)

# 3. On trie le dataframe et on réinitialise l'index
data_BRICSAM = data_BRICSAM.sort_values(by=['Pays', 'Indicateur', 'Année'])
data_BRICSAM = data_BRICSAM.reset_index(drop=True)

# 4. On exporte en CSV
data_BRICSAM.to_csv("données_BRICSAM.csv")

print(data_BRICSAM.head(66))

      Pays                                         Indicateur  Année    Unité  \
0   Brazil                                   PIB par habitant   2000        $   
1   Brazil                                   PIB par habitant   2001        $   
2   Brazil                                   PIB par habitant   2002        $   
3   Brazil                                   PIB par habitant   2003        $   
4   Brazil                                   PIB par habitant   2004        $   
..     ...                                                ...    ...      ...   
61  Brazil                   disponibilité protéiques moyenne   2017  g/cap/d   
62  Brazil                   disponibilité protéiques moyenne   2018  g/cap/d   
63  Brazil                   disponibilité protéiques moyenne   2019  g/cap/d   
64  Brazil                   disponibilité protéiques moyenne   2020  g/cap/d   
65  Brazil  suffisance des apports énergétiques alimentair...   2000        %   

         Valeur  
0   6817.

pivot

In [ ]:
def pivot_bricsam_data(df):
    """
    Transforme le dataframe pour mettre chaque indicateur en colonne 
    avec son unité intégrée dans le nom.
    """
    # 1. On crée le nouveau nom de colonne : "Indicateur (Unité)"
    df['Indicateur_Unite'] = df['Indicateur'] + " (" + df['Unité'] + ")"
    
    # 2. On pivote le tableau
    # index : ce qui définit une ligne unique (Pays + Année)
    # columns : ce qui devient les nouvelles colonnes
    # values : ce qui remplit les cases
    df_pivot = df.pivot(index=['Pays', 'Année'], 
                        columns='Indicateur_Unite', 
                        values='Valeur').reset_index()
    
    # 3. On trie par Pays et Année pour la clarté
    df_pivot = df_pivot.sort_values(by=['Pays', 'Année'])
    
    return df_pivot



In [82]:
# Utilisation
data_BRICSAM_pivot = pivot_bricsam_data(data_BRICSAM)

data_BRICSAM_pivot.to_csv("données_BRICSAM_pivot.csv")


## Partie 2 : DATA VISUALIZATION

In [85]:
data_BRICSAM_pivot.head(10)

Indicateur_Unite,Pays,Année,PIB par habitant ($),disponibilité alimentaire par habitant (kcal/cap/d),disponibilité protéiques moyenne (g/cap/d),suffisance des apports énergétiques alimentaires moyens (%)
0,Brazil,2000,6817.783456,2896.0,80.2,122.0
1,Brazil,2001,6823.033185,2960.0,81.8,124.0
2,Brazil,2002,6944.622576,3031.0,83.2,127.0
3,Brazil,2003,6941.439659,3100.0,83.8,129.0
4,Brazil,2004,7258.781017,3133.0,83.7,130.0
5,Brazil,2005,7409.568280,3167.0,84.2,131.0
6,Brazil,2006,7622.039194,3201.0,86.1,132.0
7,Brazil,2007,8003.253210,3235.0,87.5,133.0
8,Brazil,2008,8330.763640,3269.0,92.5,134.0
9,Brazil,2009,8246.096121,3307.0,97.0,136.0


### Evolutions graphiques

Evolutions des PIB/HAB

In [ ]:
for pays in data_BRICSAM_pivot['Pays'].unique() :
    PIB_hab_pays=data_BRICSAM_pivot.loc[data_BRICSAM_pivot['Pays'] == pays]
    plt.plot(PIB_hab_pays['Pays'],PIB_hab_pays['PIB_par_habitant'],label=pays)

plt.xlabel('Année')    
plt.ylabel('PIB par habitant ($ US courants)')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

NameError: name 'PIB_hab_BRICSAM' is not defined

Illustrations indicateurs

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='suffisance des apports énergétiques alimentaires moyens']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('suffisance des apports énergétiques alimentaires moyens')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='disponibilité alimentaire par habitant']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('disponibilité alimentaire par habitant')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

In [ ]:
for pays in data_faostat['Pays'].unique() :
    data = data_faostat.loc[data_faostat['Indicateur']=='disponibilité protéiques moyenne']
    data_pays = data.loc[data['Pays']==pays]
    plt.plot(data_pays['Année'],data_pays['Valeur'],label=pays)

plt.xlabel('Année')    
plt.ylabel('disponibilité protéiques moyenne')
plt.grid()
plt.legend()
plt.xlim(left=2000, right=2025)
plt.show()

## Partie 3 : MODELE

In [84]:
data_BRICSAM_pivot.head(10)

Indicateur_Unite,Pays,Année,PIB par habitant ($),disponibilité alimentaire par habitant (kcal/cap/d),disponibilité protéiques moyenne (g/cap/d),suffisance des apports énergétiques alimentaires moyens (%)
0,Brazil,2000,6817.783456,2896.0,80.2,122.0
1,Brazil,2001,6823.033185,2960.0,81.8,124.0
2,Brazil,2002,6944.622576,3031.0,83.2,127.0
3,Brazil,2003,6941.439659,3100.0,83.8,129.0
4,Brazil,2004,7258.781017,3133.0,83.7,130.0
5,Brazil,2005,7409.568280,3167.0,84.2,131.0
6,Brazil,2006,7622.039194,3201.0,86.1,132.0
7,Brazil,2007,8003.253210,3235.0,87.5,133.0
8,Brazil,2008,8330.763640,3269.0,92.5,134.0
9,Brazil,2009,8246.096121,3307.0,97.0,136.0


### ACP

### Régression(s) linéaire PIB/hab

### Régression(s) linéaire IDH